# Importando bibliotecas

In [2]:
import pandas as pd
import os
# instalar openpyxl para leitura dos arquivos excel (https://stackoverflow.com/a/56992903)

# Importando dados
Construí paths relativos independentes do sistema operacional para facilitar o uso por outros usuários.

In [3]:
# Lendo arquivos excel

# Obtendo o path deste notebook
current_dir = os.getcwd()

# Construindo o path para os arquivos excel
itens_path = os.path.join(current_dir, 'dados', 'itens.xlsx')
pedido_path = os.path.join(current_dir, 'dados', 'pedido.xlsx')
itens_pedido_path = os.path.join(current_dir, 'dados', 'itens_pedido.xlsx')

# Importando arquivos excel como dataframes
df_itens = pd.read_excel(itens_path)
df_pedido = pd.read_excel(pedido_path)
df_itens_pedido = pd.read_excel(itens_pedido_path)

## Preparação dos dataframes
1. Renomear colunas de df_itens: 'ID_ITEM' e 'PRECO_ITEM'
2. Dropar colunas inúteis de df_pedido e df_itens_pedido: 'Unnamed: 0'
3. Adicionar coluna 'PRECO_ITEM' na tabela 'df_itens_pedido': utilize a tabela df_itens para isso (join?)
4. Adicionar coluna 'PRECO_TOTAL_ITEM' na tabela df_itens_pedido: ela vai multiplicar PRECO_ITEM por QUANTIDADE
5. Adicionar coluna 'DATA' na tabela df_itens_pedido: utilizar a tabela df_pedido (join?)
6. PARA DESCOBRIR O PREÇO TOTAL POR PEDIDO:
    1. Agrupar a tabela df_itens_pedido por ID_PEDIDO, gerando uma nova tabela com as seguintes colunas:
        - ID_PEDIDO
        - DATA
        - PRECO_PEDIDO: é a soma das linhas em PRECO_TOTAL_ITEM para cada grupo de ID_PEDIDO
    2. Essa nova tabela pode se chamar df_pedidos


In [ ]:
# renomear colunas de df_itens
df_itens = df_itens.rename(columns={
    'Unnamed: 0': 'ID_ITEM',
    0: 'PRECO_ITEM'
})

In [ ]:
# descartar colunas Unnamed de df_pedido e df_itens_pedido
df_pedido = df_pedido.drop(columns='Unnamed: 0')
df_itens_pedido = df_itens_pedido.drop(columns='Unnamed: 0')

In [ ]:
# adicionar 'PRECO_ITEM' na tabela 'df_itens_pedido' utilizando a tabela 'df_itens'
df_itens_pedido = df_itens_pedido.merge(df_itens, on='ID_ITEM')

In [ ]:
# adicionar 'PRECO_TOTAL_ITEM' em 'df_itens_pedido'
df_itens_pedido['PRECO_TOTAL_ITEM'] = df_itens_pedido['QUANTIDADE'] * df_itens_pedido['PRECO_ITEM']
df_itens_pedido.head()

In [18]:
# adicionar coluna 'DATA' em 'df_itens_pedido' utilizando 'df_pedido'
df_itens_pedido = df_itens_pedido.merge(df_pedido, on='ID_PEDIDO')
df_itens_pedido = df_itens_pedido.drop(columns='VALOR_TOTAL')
# df_itens_pedido.head()

,ID_PEDIDO,ID_ITEM,QUANTIDADE,PRECO_ITEM,PRECO_TOTAL_ITEM,DATA
0,120210601,item C,4,20,80,2021-06-01
1,220210601,item A,3,10,30,2021-06-01
2,320210604,item C,1,20,20,2021-06-04
3,420210604,item B,4,15,60,2021-06-04
4,420210604,item C,2,20,40,2021-06-04


In [26]:
# verificando se cada pedido ocorre apenas em uma única data
unique_dates = df_pedido.groupby(by='ID_PEDIDO')['DATA'].nunique()

pedidos_varias_datas = unique_dates[unique_dates > 1]

if not pedidos_varias_datas.empty:
    print("Pedidos com múltiplas datas:")
    print(pedidos_varias_datas)
else:
    print("Todos os pedidos têm apenas datas diferentes")


# criar nova tabela agrupando todosos dados por pedido, exibindo o valor total por pedido = soma de 'PRECO_TOTAL_ITEM' por ID_PEDIDO
df_pedidos = df_itens_pedido.groupby(by='ID_PEDIDO', as_index=False).agg({
    'PRECO_TOTAL_ITEM':'sum',
    'DATA': 'first'
})
df_pedidos = df_pedidos.rename(columns={'PRECO_TOTAL_ITEM':'VALOR_PEDIDO'})
df_pedidos.head()

,ID_PEDIDO,VALOR_PEDIDO,DATA
0,120210601,80,2021-06-01
1,220210601,30,2021-06-01
2,320210604,20,2021-06-04
3,420210604,100,2021-06-04
4,520210604,30,2021-06-04


Agora temos duas tabelas importantes no projeto:

- *df_itens_pedido*, com informações detalhadas de itens por pedido, com preço do item, preço total, data
- *df_pedidos*, com informações consolidadas dos pedidos feitos, com o valor total do pedido (que pode ter vários itens) e a data do pedido.

# Análise Exploratória (EDA)
- Nomes de colunas
- Dimensões
- Tipos de dados
- Estatística descritiva: 
    - Separar em dados categóricos e numéricos
    - Analisar a distribuição dos dados
    - Analisar a correlação dos dados (matriz de correlação): atenção porque nem todas as variáveis podem ser correlacionadas
    - Verificar se existe sazonalidade

# Testes de Hipóteses
A partir da análise exploratória, levantar algumas hipóteses para obter insights dos dados. No contexto deste teste talvez esta etapa não será necessária.

# Featuring Engineering
- Preparar e transformar os dados para o treinamento dos modelos de machine learning
    - Standardization
    - Rescaling
    - Encoding
    - Features Selection (seleção de atributos): se o número de atributos for grande, podemos utilizar um modelo para selecioná-los, como o Boruta.
- Tratamento de dados (nomes de colunas, linhas vazias, duplicadas)

(*ver https://towardsdatascience.com/lets-do-feature-engineering-5731efc3d7fe*)

# Modelagem
1. Identificar o tipo de problema: previsão (forecasting)
2. Identificar tipos de modelos de machine learning adequados: para previsão são estes:
    - Modelos de Time Series (ARIMA, SARIMA, Suavizações Exponenciais)
    - Modelos de Regressão (linear, random forest, gradient boosting (XGBoost))
3. Escolher modelo baseline: o mais simples possível.
4. Separar dados em treino e teste
5. Coletar performance para cada modelo escolhido: avaliar métricas adequadas p/os modelos escolhidos
    - MAE
    - MAPE
    - RMSE
    - Forecast Bias
    - R-squared
6. Escolhido o modelo, fazer o *fine tunning* dele (cuidado com overfitting)
7. Coletar a performance do modelo otimizado (métricas e gráfico de previsão x resultado real) e salvar o modelo para não precisar treiná-lo novamente.